In [ ]:
import sys

sys.path.append('..')


from torch.utils.data import DataLoader
import pandas as pd
from pathlib import Path
import numpy as np
from torch import nn

from src.trainer import Trainer
from src.dataset import HumanPosesDataset
from sklearn.model_selection import train_test_split
import torch

In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

In [ ]:
from src.augmentations import StrongAugment, TestAugment

# ВНИМАНИЕ: там поменялись аугментации с момента обучения ефишент нета

train_transform = StrongAugment(image_size=224,
                                use_autoaug=True,
                                use_random_erasing=True)

val_transform = TestAugment(image_size=224)

In [ ]:
CSV_PATH = Path("../data/human_poses_data/train_answers.csv")
TRAIN_DIR = Path("../data/human_poses_data/img_train")

df = pd.read_csv(CSV_PATH)

train_ids, val_ids = train_test_split(
    df['img_id'].values,
    test_size=0.2,
    stratify=df['target_feature'],
    random_state=42
)

train_df = df[df['img_id'].isin(train_ids)].reset_index(drop=True)
val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)

train_dataset = HumanPosesDataset(
    data_df=train_df,
    img_dir=TRAIN_DIR,
    transform=train_transform,
)

val_dataset = HumanPosesDataset(
    data_df=val_df,
    img_dir=TRAIN_DIR,
    transform=val_transform,
)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

In [ ]:
num_classes = len(np.unique(df['target_feature']))
print(f"Количество классов: {num_classes}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

In [ ]:
from src.models.enet import EfficientNetV2

model = EfficientNetV2(variant='M', num_classes=num_classes).to(device)

In [ ]:
import torch
from ranger21 import Ranger21
from torch.amp import GradScaler

NUM_EPOCH   = 50
BATCH_SIZE   = 48
BASE_LR      = 3e-4
WEIGHT_DECAY = 0.05

optimizer = Ranger21(
    model.parameters(),
    lr=BASE_LR,
    weight_decay=WEIGHT_DECAY,
    betas=(0.9, 0.999),
    use_warmup=False,
    warmdown_active=False,
    decay_type=None,
    num_epochs=NUM_EPOCH,
    num_batches_per_epoch=len(train_loader)
)


scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=BASE_LR,
    epochs=NUM_EPOCH,
    steps_per_epoch=len(train_loader),
    pct_start=0.10,
    div_factor=25.0,
    final_div_factor=1e4,
    anneal_strategy='cos'
)

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
scaler    = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=0.75, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    scaler=scaler,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="sota_enet_2_1",
    use_wandb=True,
    seed=42,
)

history = trainer.train()

In [ ]:
from src.utils import load_best_model

load_best_model(model, 'checkpoints/sota_enet_2_1_best.pth', device)

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 75

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    betas=(0.9, 0.999),
    weight_decay=0.05
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler    = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=0.75, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    scaler=scaler,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="sota_enet_2_2",
    use_wandb=True,
    seed=42,
)

history = trainer.train()

In [ ]:
load_best_model(model, 'checkpoints/sota_enet_2_2_best.pth', device)

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 75

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    betas=(0.9, 0.999),
    weight_decay=0.05
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler    = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=0.75, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    scaler=scaler,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="sota_enet_2_3",
    use_wandb=True,
    seed=42,
)

history = trainer.train()

In [ ]:
load_best_model(model, 'checkpoints/sota_enet_2_3_best.pth', device)

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 75

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    betas=(0.9, 0.999),
    weight_decay=0.05
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler    = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=0.5, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    scaler=scaler,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="sota_enet_2_4",
    use_wandb=True,
    seed=42,
)

history = trainer.train()

In [ ]:
load_best_model(model, 'checkpoints/sota_enet_2_4_best.pth', device)

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 75

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    betas=(0.9, 0.999),
    weight_decay=0.05
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler    = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=0.5, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    scaler=scaler,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="sota_enet_2_5",
    use_wandb=True,
    seed=42,
)

history = trainer.train()